In [1]:
#importing necessary libraries

import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from warnings import warn
import time

In [11]:
#passing required URL for scrapping

driver=webdriver.Edge('msedgedriver.exe')
driver.get("https://www.linkedin.com")
driver.maximize_window()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_1736\4130809575.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Edge('msedgedriver.exe')


In [12]:
#logging in using keys

inputID = driver.find_element(by=By.ID, value = "session_key")
inputPass = driver.find_element(by=By.ID, value = "session_password")
signIn = driver.find_element(by=By.CLASS_NAME, value = "sign-in-form__submit-button ")
inputID.send_keys("durgeshkumar7b@gmail.com")
inputPass.send_keys("@Durgesh01")
signIn.click()

time.sleep(10)

# login_btn = driver.find_element(By.CLASS_NAME,"sign-in-form__submit-button")
# login_btn.click()

In [13]:
#redirecting to desired URL

driver.get("https://www.linkedin.com/jobs/collections/")

In [14]:
#list of elements required

name = []
designation = []
location = []
job_link = []
industry = []
emp_count = []
linkedin_followers = []
applicants = []
involvement = []
work_type = []

In [15]:
#iterating through page

for i in range(1,41):
    #button path for page numbers
    path ='//button[@aria-label="Page {}"]'.format(i)
    
    #button clicking    
    driver.find_element(By.XPATH, path).click()    
    
    #html data
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')
    
    #main page of one job data
    lk=soup.findAll(class_="disabled ember-view job-card-container__link")
     
    #link of a single job data
    for i in lk:
        # links
        li=i['href']
        
        #every page data
        every_page =BeautifulSoup(driver.page_source,'lxml')
                
        #movig to link using next window_of_ chrome -- alternative of redirecting to original URL
        driver.switch_to.new_window('tab')
        job_link.append("https://www.linkedin.com{}".format(li))
        driver.get("https://www.linkedin.com{}".format(li))
        
       
         # company name
        try:
            c_name = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__company-name')
            name.append(c_name[0].text) 
        except:
            name.append("N.A.")
         
        #designation   
        try:
            d = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__job-title')
            designation.append(d[0].text)
        except:
            designation.append("N.A.")
        
        #applicants
        try:
            apl= driver.find_elements(By.XPATH,'/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[1]/span[2]/span[2]/span')
            applicants.append(apl[0].text) 
        except:
            applicants.append("0") 
            
            
            
        #work type
        try:
            w = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__workplace-type')
            work_type.append(w[0].text)  
        except:
            work_type.append("N.A.")
        
        
        #involvement
        try:
            inv = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__job-insight')
            involvement.append(inv[0].text)
        except:
            involvement.append("N.A.")
         
        #employee count
        try:
            emp = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__job-insight')
            emp_count.append(emp[1].text)
        except:
            emp_count.append("N.A.")
        
        #location
        try:
            loc = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__bullet')
            location.append(loc[0].text)
        except:
            location.append("N.A.")
    
        #every page data
        every_page =BeautifulSoup(driver.page_source,'lxml')
        
        
        # details
        s = []
        src = driver.page_source
        soup = BeautifulSoup(src, 'lxml')
        detail = soup.findAll(class_='ember-view t-black t-normal')
        for z in detail:
            s.append(z)

        # selecting new jobs 
        for i in s:
            pr = i['href']
            
            #movig to link using next window_of_ chrome
            driver.switch_to.new_window('tab')
            driver.get("https://www.linkedin.com{}".format(pr))
            
            time.sleep(6)
            
            #industry
            try:
                ind = driver.find_elements(By.CLASS_NAME,'org-top-card-summary-info-list__info-item')
                industry.append(ind[0].text)
            except:
                industry.append("not specify")

            #followers    
            try:
                follow = driver.find_elements(By.XPATH,'//*[@id="ember28"]/div[2]/div[1]/div[1]/div[2]/div/div/div[2]/div[2]')
                linkedin_followers.append(follow[0].text)
            except:
                linkedin_followers.append("N/A")
                
                
            #close current window       
            driver.close()
            #switch to main(starting) tab/window
            driver.switch_to.window(driver.window_handles[-1])
     
               
        # close current window       
        driver.close()
        #switch to main (starting) tab/window
        driver.switch_to.window(driver.window_handles[0])


In [16]:
#checking length of lists 

len(name), len(location), len(applicants), len(designation),len(emp_count),len(industry),len(linkedin_followers),len(involvement),len(work_type)

(283, 283, 283, 283, 283, 248, 248, 283, 283)

In [25]:
# if length are not equal
if len(name) != len(industry):
    # Append mean values to the list with smaller length
    if len(name) > len(industry):
        alpha = np.nan
        industry += (len(name)-len(industry)) * [alpha]
    else:
        pass

In [ ]:
# if length are not equal
if len(name) != len(linkedin_followers):
    # Append mean values to the list with smaller length
    if len(name) > len(linkedin_followers):
        beta = np.nan
        linkedin_followers += (len(name)-len(linkedin_followers)) * [beta]
    else:
        pass

In [26]:
#creating tables using pandas

main_table = pd.DataFrame({'name':name,'employees_count':emp_count,'linkedin_followers':linkedin_followers,
                           'industry':industry,'involvement':involvement,'work_type':work_type ,'total_applicants':applicants,
                           'location':location, 'designation':designation})

In [27]:
import openpyxl

main_table.to_excel('main_table.xlsx', sheet_name='sheet_1')

In [29]:
#CLeaning
import pandas as pd
import numpy as np

In [58]:
df=pd.read_excel('main_table.xlsx')

In [71]:
df.head()

,name,employees_count,linkedin_followers,industry,involvement,work_type,total_applicants,location,designation
0,Aces Global Consulting Pvt Ltd,11-50 employees · Human Resources Services,681 followers,Human Resources Services,Full-time · Entry level,On-site,2 applicants,"New Delhi, Delhi, India",MIS Executive & Coordinator
1,Cyient,"10,001+ employees · IT Services and IT Consulting","314,683 followers",IT Services and IT Consulting,Full-time · Entry level,Hybrid,0,"Bengaluru, Karnataka, India",SQL developer
2,Creditsafe Technology,51-200 employees · Information Technology & Se...,"11,160 followers",Information Technology & Services,Full-time · Mid-Senior level,Hybrid,0,"Hyderabad, Telangana, India",Data Analyst
3,AIMLEAP,51-200 employees,"11,575 followers",IT Services and IT Consulting,Full-time,Remote,0,India,Web Scraping Specialist/Data Mining Analyst- W...
4,Aditya Anagha Multi State Credit Co-Operative ...,201-500 employees,218 followers,Banking,Full-time,On-site,31 applicants,"Nagpur, Maharashtra, India",Data Analyst


In [61]:
df.isna().sum()

name                   0
employees_count        0
linkedin_followers    75
industry              35
involvement            0
work_type              0
total_applicants       0
location               0
designation            0
dtype: int64

In [72]:
for i in df['employees_count']:
    i=i.split(' ')[0]
    print(i)
    

11-50
10,001+
51-200
51-200
201-500
51-200
11-50
51-200
1,001-5,000
10,001+
1-10
10,001+
10,001+
11-50
51-200
1,001-5,000
51-200
NISARG
10,001+
10,001+
51-200
51-200
10,001+
11-50
N.A.
11-50
11-50
1-10
11-50
1,001-5,000
11-50
11-50
1,001-5,000
N.A.
1-10
51-200
51-200
5,001-10,000
Skills:
501-1,000
11-50
51-200
N.A.
51-200
11-50
10,001+
51-200
10,001+
501-1,000
1-10
51-200
10,001+
1,001-5,000
10,001+
1-10
1,001-5,000
N.A.
51-200
51-200
51-200
11-50
1-10
51-200
N.A.
51-200
N.A.
51-200
51-200
10,001+
1-10
201-500
51-200
1,001-5,000
10,001+
51-200
1,001-5,000
201-500
10,001+
1,001-5,000
Skills:
11-50
Skills:
10,001+
11-50
10,001+
10,001+
10,001+
Skills:
10,001+
10,001+
10,001+
11-50
11-50
501-1,000
1,001-5,000
11-50
N.A.
1,001-5,000
10,001+
51-200
11-50
10,001+
501-1,000
1-10
1-10
51-200
1,001-5,000
11-50
11-50
1-10
10,001+
N.A.
51-200
N.A.
1-10
5,001-10,000
51-200
N.A.
201-500
11-50
11-50
11-50
11-50
10,001+
11-50
N.A.
10,001+
501-1,000
51-200
10,001+
501-1,000
201-500
10,001+
11-50
51-20

In [51]:
df['employees_count'].replace(np.nan,'0, empo',inplace=True)

In [52]:
df['linkedin_followers'].replace(np.nan,'0, empo',inplace=True)

In [57]:
df['employees_count'].str

In [55]:
df.head()

,name,employees_count,linkedin_followers,industry,involvement,work_type,total_applicants,location,designation
0,Aces Global Consulting Pvt Ltd,"[11-50, employees, ·, Human, Resources, Services]",681 followers,Human Resources Services,Full-time · Entry level,On-site,2 applicants,"New Delhi, Delhi, India",MIS Executive & Coordinator
1,Cyient,"[10,001+, employees, ·, IT, Services, and, IT,...","314,683 followers",IT Services and IT Consulting,Full-time · Entry level,Hybrid,0,"Bengaluru, Karnataka, India",SQL developer
2,Creditsafe Technology,"[51-200, employees, ·, Information, Technology...","11,160 followers",Information Technology & Services,Full-time · Mid-Senior level,Hybrid,0,"Hyderabad, Telangana, India",Data Analyst
3,AIMLEAP,"[51-200, employees]","11,575 followers",IT Services and IT Consulting,Full-time,Remote,0,India,Web Scraping Specialist/Data Mining Analyst- W...
4,Aditya Anagha Multi State Credit Co-Operative ...,"[201-500, employees]",218 followers,Banking,Full-time,On-site,31 applicants,"Nagpur, Maharashtra, India",Data Analyst
